In [33]:
#Load dependencies
import os
import pandas as pd
import numpy as np

## Load Data

In [34]:
#Add local folder path
folder_path_5y = 'data/5Y'

#Add local filepaths
csv_files = [f for f in os.listdir(folder_path_5y) 
            if f.endswith('Data.csv')]

#Create and combine all individual dataframes
dfs = []

for file in csv_files:
    file_path = os.path.join(folder_path_5y, file)
    
    df = pd.read_csv(file_path, header=0)
    
    df['source_file'] = file
    
    dfs.append(df)

combined_df = pd.concat(dfs, axis=0, ignore_index=True)


In [35]:
print(f"Combined DataFrame shape: {combined_df.shape}")
combined_df.head(300)

Combined DataFrame shape: (3875, 132)


,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C02_001E,S1901_C02_001M,S1901_C03_001E,S1901_C03_001M,S1901_C04_001E,S1901_C04_001M,...,S1901_C01_016E,S1901_C01_016M,S1901_C02_016E,S1901_C02_016M,S1901_C03_016E,S1901_C03_016M,S1901_C04_016E,S1901_C04_016M,Unnamed: 130,source_file
0,Geography,Geographic Area Name,Households!!Estimate!!Total,Households!!Margin of Error!!Total,Families!!Estimate!!Total,Families!!Margin of Error!!Total,Married-couple families!!Estimate!!Total,Married-couple families!!Margin of Error!!Total,Nonfamily households!!Estimate!!Total,Nonfamily households!!Margin of Error!!Total,...,Households!!Estimate!!PERCENT IMPUTED!!Nonfami...,Households!!Margin of Error!!PERCENT IMPUTED!!...,Families!!Estimate!!PERCENT IMPUTED!!Nonfamily...,Families!!Margin of Error!!PERCENT IMPUTED!!No...,Married-couple families!!Estimate!!PERCENT IMP...,Married-couple families!!Margin of Error!!PERC...,Nonfamily households!!Estimate!!PERCENT IMPUTE...,Nonfamily households!!Margin of Error!!PERCENT...,NaN,ACSST5Y2011.S1901-Data.csv
1,0400000US49,Utah,871358,2763,654354,3519,536979,3904,217004,2609,...,(X),(X),(X),(X),(X),(X),23.5,(X),NaN,ACSST5Y2011.S1901-Data.csv
2,8600000US81324,ZCTA5 81324,572,84,415,57,359,60,157,58,...,(X),(X),(X),(X),(X),(X),24.8,(X),NaN,ACSST5Y2011.S1901-Data.csv
3,8600000US82930,ZCTA5 82930,5144,296,3706,303,2967,305,1438,240,...,(X),(X),(X),(X),(X),(X),16.4,(X),NaN,ACSST5Y2011.S1901-Data.csv
4,8600000US83312,ZCTA5 83312,45,39,45,39,45,39,0,89,...,(X),(X),(X),(X),(X),(X),-,(X),NaN,ACSST5Y2011.S1901-Data.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,8600000US86514,ZCTA5 86514,770,85,542,72,268,50,228,62,...,(X),(X),(X),(X),(X),(X),17.1,(X),NaN,ACSST5Y2011.S1901-Data.csv
296,Geography,Geographic Area Name,Households!!Estimate!!Total,Households!!Margin of Error!!Total,Families!!Estimate!!Total,Families!!Margin of Error!!Total,Married-couple families!!Estimate!!Total,Married-couple families!!Margin of Error!!Total,Nonfamily households!!Estimate!!Total,Nonfamily households!!Margin of Error!!Total,...,Households!!Estimate!!PERCENT IMPUTED!!Nonfami...,Households!!Margin of Error!!PERCENT IMPUTED!!...,Families!!Estimate!!PERCENT IMPUTED!!Nonfamily...,Families!!Margin of Error!!PERCENT IMPUTED!!No...,Married-couple families!!Estimate!!PERCENT IMP...,Married-couple families!!Margin of Error!!PERC...,Nonfamily households!!Estimate!!PERCENT IMPUTE...,Nonfamily households!!Margin of Error!!PERCENT...,NaN,ACSST5Y2012.S1901-Data.csv
297,0400000US49,Utah,880873,2335,659829,3220,538595,3885,221044,2613,...,(X),(X),(X),(X),(X),(X),24.3,(X),NaN,ACSST5Y2012.S1901-Data.csv
298,8600000US81324,ZCTA5 81324,518,81,360,59,315,59,158,54,...,(X),(X),(X),(X),(X),(X),33.5,(X),NaN,ACSST5Y2012.S1901-Data.csv


## Clean Data

#### Handle Missing Data

In [36]:
#Different values to set as NA for filtering of columns and imputation later
na_strings_list = [
    "**",  #Margin of error column - no sample or too few samples for standard error
    "-",   #Estimate column - no sample or too few samples for estimate or ratio of medians issue
    "-",   #Following median estimate - median falls in lowest interval of open-ended distribution
    "+",   #Following median estimate - median falls in upper interval of open-ended distribution
    "***", #Margin of error column - median in lowest/upper interval of open-ended distribution
    "*****", #Margin of error column - estimate is controlled
    "N",   #Estimate and margin of error columns - data cannot be displayed (sample too small)
    "(X)"  #Estimate not applicable or not available
]

In [37]:
#Replace special character values with np.nan values
cleaned_df = combined_df.replace(na_strings_list, np.nan)

#Check nan value counts
missing_counts = cleaned_df.isna().sum().sort_values(ascending=False)
print(missing_counts)

#Filter out columns with >200 missing values (arbitrary decision, would consider more if there was a specific use case for this project)
na_columns = missing_counts[missing_counts > 200].index.tolist()

cleaned_df = cleaned_df.drop(columns=na_columns)
cleaned_df.head(5)

Unnamed: 130      3875
S1901_C02_016E    3862
S1901_C01_015M    3862
S1901_C03_016E    3862
S1901_C02_015M    3862
                  ... 
S1901_C03_001E       0
S1901_C02_001E       0
S1901_C02_001M       0
S1901_C03_001M       0
source_file          0
Length: 132, dtype: int64


,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C02_001E,S1901_C02_001M,S1901_C03_001E,S1901_C03_001M,S1901_C04_001E,S1901_C04_001M,...,S1901_C01_011M,S1901_C02_011E,S1901_C02_011M,S1901_C03_011E,S1901_C03_011M,S1901_C01_013E,S1901_C01_013M,S1901_C01_014E,S1901_C02_015E,source_file
0,Geography,Geographic Area Name,Households!!Estimate!!Total,Households!!Margin of Error!!Total,Families!!Estimate!!Total,Families!!Margin of Error!!Total,Married-couple families!!Estimate!!Total,Married-couple families!!Margin of Error!!Total,Nonfamily households!!Estimate!!Total,Nonfamily households!!Margin of Error!!Total,...,"Households!!Margin of Error!!$200,000 or more","Families!!Estimate!!$200,000 or more","Families!!Margin of Error!!$200,000 or more","Married-couple families!!Estimate!!$200,000 or...",Married-couple families!!Margin of Error!!$200...,Households!!Estimate!!Mean income (dollars),Households!!Margin of Error!!Mean income (doll...,Households!!Estimate!!PERCENT IMPUTED!!Househo...,Families!!Estimate!!PERCENT IMPUTED!!Family in...,ACSST5Y2011.S1901-Data.csv
1,0400000US49,Utah,871358,2763,654354,3519,536979,3904,217004,2609,...,0.1,4.1,0.1,4.7,0.2,72305,436,23.6,22.9,ACSST5Y2011.S1901-Data.csv
2,8600000US81324,ZCTA5 81324,572,84,415,57,359,60,157,58,...,5.5,0.0,7.5,0.0,8.6,50057,7443,29.9,31.8,ACSST5Y2011.S1901-Data.csv
3,8600000US82930,ZCTA5 82930,5144,296,3706,303,2967,305,1438,240,...,1.2,2.4,1.7,3.0,2.1,69452,6380,18.7,19.1,ACSST5Y2011.S1901-Data.csv
4,8600000US83312,ZCTA5 83312,45,39,45,39,45,39,0,89,...,44.2,0.0,44.2,0.0,44.2,65527,28396,35.6,35.6,ACSST5Y2011.S1901-Data.csv


In [38]:
#Check nan value counts again
missing_counts = cleaned_df.isna().sum().sort_values(ascending=False)
print(missing_counts)

S1901_C01_013M    142
S1901_C01_013E    142
S1901_C03_009M     85
S1901_C03_003E     85
S1901_C03_003M     85
                 ... 
S1901_C03_001E      0
S1901_C02_001E      0
S1901_C02_001M      0
S1901_C01_001M      0
source_file         0
Length: 75, dtype: int64


#### Remove Duplicate Rows

In [39]:
#Remove duplicate rows (while accounting for minor difference due to source_file column)
non_id_list = list(cleaned_df.columns)
non_id_list.remove('source_file')

print(cleaned_df.duplicated(subset=non_id_list).sum())
cleaned_df.drop_duplicates(keep='first', inplace=True)

52


#### Replace Column Header

In [40]:
#Set column headers to more detailed description
cleaned_df.columns = cleaned_df.iloc[0]
cleaned_df = cleaned_df.iloc[1:].reset_index(drop=True)

#### Remove Aggregated Utah Statistics

In [41]:
#Get rid of aggregated Utah rows (we only want zip code level data)
cleaned_df = cleaned_df[cleaned_df['Geographic Area Name'] != 'Utah']

#### Create Zip Code and Year Variables

In [42]:
#Create zip_code column
cleaned_df['zip_code'] = cleaned_df['Geographic Area Name'].str[-5:]

#Create year column
cleaned_df['year'] = cleaned_df['ACSST5Y2011.S1901-Data.csv'].str.extract(r'Y(\d{4})\.', expand=False).astype(int)

#### Cast Dataframe Numeric Variables

In [43]:
#Convert dataframe numeric variables to numeric datatypes
numeric_columns = list(cleaned_df.columns)
string_columns = ['Geography', 'Geographic Area Name', 'ACSST5Y2011.S1901-Data.csv']
numeric_columns = [x for x in numeric_columns if x not in string_columns]

cleaned_df[numeric_columns] = cleaned_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

#### Impute Remaining Missing Values Based on Column Median

In [44]:

def impute_missing_values(df):
    #Separate numeric and non-numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns

    fill_values = {}

    #Impute numeric columns with median (robust to outliers)
    for col in numeric_cols:
        fill_values[col] = df[col].median()

    #Fill missing values
    return df.fillna(fill_values)

In [45]:
imputed_df = impute_missing_values(cleaned_df)

imputed_df.head(imputed_df.shape[0])

,Geography,Geographic Area Name,Households!!Estimate!!Total,Households!!Margin of Error!!Total,Families!!Estimate!!Total,Families!!Margin of Error!!Total,Married-couple families!!Estimate!!Total,Married-couple families!!Margin of Error!!Total,Nonfamily households!!Estimate!!Total,Nonfamily households!!Margin of Error!!Total,...,"Families!!Margin of Error!!$200,000 or more","Married-couple families!!Estimate!!$200,000 or more","Married-couple families!!Margin of Error!!$200,000 or more",Households!!Estimate!!Mean income (dollars),Households!!Margin of Error!!Mean income (dollars),Households!!Estimate!!PERCENT IMPUTED!!Household income in the past 12 months,Families!!Estimate!!PERCENT IMPUTED!!Family income in the past 12 months,ACSST5Y2011.S1901-Data.csv,zip_code,year
1,8600000US81324,ZCTA5 81324,572.0,84.0,415.0,57.0,359.0,60.0,157.0,58.0,...,7.5,0.0,8.6,50057.0,7443.0,29.9,31.8,ACSST5Y2011.S1901-Data.csv,81324.0,2011
2,8600000US82930,ZCTA5 82930,5144.0,296.0,3706.0,303.0,2967.0,305.0,1438.0,240.0,...,1.7,3.0,2.1,69452.0,6380.0,18.7,19.1,ACSST5Y2011.S1901-Data.csv,82930.0,2011
3,8600000US83312,ZCTA5 83312,45.0,39.0,45.0,39.0,45.0,39.0,0.0,89.0,...,44.2,0.0,44.2,65527.0,28396.0,35.6,35.6,ACSST5Y2011.S1901-Data.csv,83312.0,2011
4,8600000US83342,ZCTA5 83342,456.0,83.0,367.0,89.0,358.0,90.0,89.0,43.0,...,8.5,0.0,8.7,50413.0,9122.0,24.1,21.0,ACSST5Y2011.S1901-Data.csv,83342.0,2011
5,8600000US84001,ZCTA5 84001,271.0,96.0,233.0,97.0,200.0,87.0,38.0,24.0,...,13.0,0.0,15.0,77058.0,17258.0,60.5,65.2,ACSST5Y2011.S1901-Data.csv,84001.0,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3869,860Z200US84783,ZCTA5 84783,478.0,137.0,334.0,111.0,334.0,111.0,144.0,101.0,...,15.9,10.8,15.9,110779.0,38616.0,13.6,11.7,ACSST5Y2023.S1901-Data.csv,84783.0,2023
3870,860Z200US84784,ZCTA5 84784,220.0,60.0,220.0,60.0,119.0,53.0,0.0,12.0,...,12.2,33.6,18.5,101427.0,31811.0,37.3,37.3,ACSST5Y2023.S1901-Data.csv,84784.0,2023
3871,860Z200US84790,ZCTA5 84790,18477.0,913.0,13910.0,784.0,11586.0,686.0,4567.0,629.0,...,2.9,16.3,3.6,111536.0,7509.0,36.2,34.6,ACSST5Y2023.S1901-Data.csv,84790.0,2023
3872,860Z200US86044,ZCTA5 86044,1134.0,126.0,711.0,109.0,342.0,77.0,423.0,99.0,...,4.5,10.2,9.1,48096.0,7773.0,26.7,31.5,ACSST5Y2023.S1901-Data.csv,86044.0,2023


## Export Processed Data

In [ ]:
#Optional Export to csv
#imputed_df.to_csv('2011_2023_poverty_data_utah_by_zip.csv')